## Regression analysis

In this notebook I want to investigate whether there are relationships between demographics and the number of facilities or employment in specific areas. 

I could also extend the analysis by including spatial data "distance to port" or "distance to airport" for example. 

I am starting by using this blog as a resource:
http://darribas.org/gds_scipy16/ipynb_md/08_spatial_regression.html

How to interpret OLS results
https://medium.com/@jyotiyadav99111/statistics-how-should-i-interpret-results-of-ols-3bde1ebeec01

I watched this video:
https://pysal.org/spreg/

### OLS method

Let's start with the following independent variables:
* Population density (continuous data) Group E - TotPeople
* Ethnicity (categorical data) Group E - European,Maori,Pacific	Asian,MidELatinAfr,Other,NZ,OtherNEC,NotElseIncl	
* Income per capita (categorical data)  Group D: less5k_TotInd, bet5k10k_TotInd, bet10k20k_TotInd, bet20k30k_TotInd, bet30k50k_TotInd, bet50k70k_TotInd, greater70k_TotInd
* Workforce density (continuous data) - Group F, TotPeople

And we'll test for each of these dependent variables (GroupA file):
* Wholesale facilities (F_GeogUnits)
* Retail facilities (G_GeogUnits)
* Transport, postal and storage facilities (I461_GeogUnits + I471_GeogUnits + I481_GeogUnits)
* Storage, distribution and courier (I51_GeogUnits + I53_GeogUnits)

And we want to do this for specific areas:
* Auckland Region (Auckland)
* Waikato Region (Hamilton)
* Bay of Plenty Region (Tauranga)
* Wellington Region (Wellington)
* Canterbury Region (Christchurch)

For the following time stamps (census stamps):
* 2006
* 2013
* 2018


### Create a dataframe with the necessary data

In [110]:
import pandas as pd
import math
import numpy as np

def readFiles(input_filename):
    df=pd.read_csv(input_filename)
    df=df.drop("Unnamed: 0",axis=1)
    #Strip all leading whitespace in Area column
    df['Area'] = df['Area'].apply(lambda x: x.strip())
    return(df)

def combine(df_A,df_D,df_E,df_F):
    ## Delete the employment columns
    df_A=df_A.drop(['TotInd_EmpCo','F_EmpCo','G_EmpCo','I461_EmpCo','I471_EmpCo','I481_EmpCo','I51_EmpCo','I53_EmpCo'],axis='columns')
    ## Combine the transport and warehousing columns
    transport=df_A[['I471_GeogUnits','I461_GeogUnits','I481_GeogUnits']]
    df_A['Trans_GeogUnits']=transport.sum(axis='columns',skipna=True)
    storage=df_A[['I51_GeogUnits','I53_GeogUnits']]
    df_A['Storage_GeogUnits']=storage.sum(axis='columns',skipna=True)
    df_A=df_A.drop(['I51_GeogUnits','I53_GeogUnits','I471_GeogUnits','I461_GeogUnits','I481_GeogUnits'],axis='columns')
    ##Remove the totals rows
    df_A=df_A.loc[df_A['Area']!="Total - New Zealand by Regional Council/SA2"]
    ## Add Workforce density
    df_A['Workforce']=np.nan
    for i in range(len(df_A)):
        df_A.iloc[i,8]=df_F.loc[(df_F['Area']==df_A.iloc[i,0])&(df_F['Year']==df_A.iloc[i,5]),'TotPeople']
    ## Add Population density and Ethnicity (as proportion)
    df_A['Population']=np.nan
    #as proportion of population
    df_A['Perc_MaoriPacific']=np.nan
    df_A['Majority_MaoriPacific']=""
    df_A['Perc_less30k']=np.nan
    for i in range(len(df_A)):
        record=df_E.loc[(df_E['Area']==df_A.iloc[i,0])&(df_E['Year']==df_A.iloc[i,5]),]
        df_A.iloc[i,9]=record['TotPeople']
        df_A.iloc[i,10]=(record['Maori']+record['Pacific'])/record['TotPeople']
        if df_A.iloc[i,10]>=0.6:
            df_A.iloc[i,11]="YES"
        else:
            df_A.iloc[i,11]="NO"
    ## Add income per capita (as proportion)
        record=df_D.loc[(df_D['Area']==df_A.iloc[i,0])&(df_D['Year']==df_A.iloc[i,5]),]
        df_A.iloc[i,12]=(record['less5k_TotInd']+record['bet5k10k_TotInd']+record['bet10k20k_TotInd']+record['bet20k30k_TotInd'])/record['totStated_TotInd']
    return(df_A)

def checkAreas(df1,df2):
    missingList=[]
    for y in [2006,2013,2018]:
        set1=df1.loc[df1['Year']==y,'Area']
        set2=df2.loc[df2['Year']==y,'Area']
        listDF1=list(set(set1)-set(set2))
        listDF2=list(set(set2)-set(set1))
        missingList.append(listDF1)
        missingList.append(listDF2)
    return(missingList)


In [111]:
groupA=readFiles("../CompleteSet_GroupA.csv")
#trim to only 2006, 2013 and 2020
groupA=groupA.loc[groupA['Year'].isin([2006,2013,2018]),]
#rename the totals rows
groupA.loc[groupA['Area']=="Total NZ by Regional Council/Statistical Area",'Area']="Total - New Zealand by Regional Council/SA2"

groupF=readFiles("../CompleteSet_GroupF.csv")
groupE=readFiles("../CompleteSet_GroupE.csv")
groupD=readFiles("../CompleteSet_GroupD.csv")

By running the function checkList, I confirmed that groups E, F, and D all have the same areas. There is (the same) discrepancy between Group A and all three other sets for all three census years. 

I then investigated the implication if we just ignore the "Oceanic, Islands, Inlets, Lakes etc" that are not included in Group A. The impact is minimal as these areas have mostly zero people, some have populations but less than 30 people. 

I thus delete those areas from E, D, and F. I also remove the remaining Oceanic regions that do occur in A

Then I rename the rows that state the totals, because there is a naming discrepancy.

In [112]:
#checkList=checkAreas(groupA,groupE)
#pd.set_option('display.max_rows', None)
#groupE.loc[(groupE['Area'].isin(checkList[1]))&(groupE['Year']==2006),['Area','TotPeople']]
#groupD.loc[(groupD['Area'].isin(checkList[1]))&(groupD['Year']==2006),['Area','totStated_TotInd']]
#groupF.loc[(groupF['Area'].isin(checkList[1]))&(groupF['Year']==2006),['Area','TotPeople']]

checkList=checkAreas(groupA,groupE)
deleteList=checkList[1]+["Oceanic Chatham Islands","Oceanic Auckland Region East","Oceanic Auckland Region West","Oceanic Bay of Plenty Region","Oceanic Canterbury Region","Oceanic Waikato Region East","Oceanic Waikato Region West"]

groupE=groupE[groupE["Area"].isin(deleteList) == False]
groupF=groupF[groupF["Area"].isin(deleteList) == False]
groupD=groupD[groupD["Area"].isin(deleteList) == False]
groupA=groupA[groupA["Area"].isin(deleteList) == False]

#Now we need to combine the dataframes
df=combine(groupA,groupD,groupE,groupF)


In [113]:
df.head()

,Area,TotInd_GeogUnits,F_GeogUnits,G_GeogUnits,ParentArea,Year,Trans_GeogUnits,Storage_GeogUnits,Workforce,Population,Perc_MaoriPacific,Majority_MaoriPacific,Perc_less30k
31,Area Outside Region,201.0,3.0,6.0,NewZealand,2006,3.0,0.0,477.0,621.0,0.603865,YES,0.530973
33,Chatham Islands,198.0,3.0,6.0,AreaOutsideRegion,2006,3.0,0.0,471.0,612.0,0.612745,YES,0.530973
34,Ross Dependency,0.0,NaN,0.0,AreaOutsideRegion,2006,0.0,0.0,0.0,NaN,NaN,NO,NaN
66,Area Outside Region,189.0,NaN,6.0,NewZealand,2013,0.0,0.0,480.0,600.0,0.565000,NO,0.366972
68,Chatham Islands,189.0,NaN,6.0,AreaOutsideRegion,2013,0.0,0.0,477.0,600.0,0.565000,NO,0.370370


### Try OLS

In [114]:
import seaborn as sns
import matplotlib.pyplot as plt
import pysal as ps
import geopandas as gpd


In [134]:
auckReg_df_2006=df.loc[(df['ParentArea']=="AucklandRegion")&(df['Year']==2006),]
auckReg_df_2006=auckReg_df_2006.dropna(axis=0,how="any")
auckReg_df_2006.columns

Index(['Area', 'TotInd_GeogUnits', 'F_GeogUnits', 'G_GeogUnits', 'ParentArea',
       'Year', 'Trans_GeogUnits', 'Storage_GeogUnits', 'Workforce',
       'Population', 'Perc_MaoriPacific', 'Majority_MaoriPacific',
       'Perc_less30k'],
      dtype='object')

In [135]:

indep=np.asarray(auckReg_df_2006[['Perc_less30k','Perc_MaoriPacific']])
y=np.asarray(auckReg_df_2006.TotInd_GeogUnits)
y.shape = (len(y), 1)
m1 = ps.model.spreg.OLS(y,indep,name_x=['Perc_less30k','Perc_MaoriPacific'], name_y='Tot_GeogUnits')

print(m1.summary)


REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Tot_GeogUnits                Number of Observations:         522
Mean dependent var  :    289.5575                Number of Variables   :           3
S.D. dependent var  :    312.2878                Degrees of Freedom    :         519
R-squared           :      0.0819
Adjusted R-squared  :      0.0784
Sum squared residual:46648641.389                F-statistic           :     23.1481
Sigma-square        :   89881.775                Prob(F-statistic)     :   2.346e-10
S.E. of regression  :     299.803                Log likelihood        :   -3716.213
Sigma-square ML     :   89365.213                Akaike info criterion :    7438.426
S.E of regression ML:    298.9402                Schwarz criterion     :    7451.199

----------------------------------------------------------------------------

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Trans_GeogUnits                Number of Observations:         522
Mean dependent var  :      3.2069                Number of Variables   :           2
S.D. dependent var  :      3.7441                Degrees of Freedom    :         520
R-squared           :      0.0101
Adjusted R-squared  :      0.0082
Sum squared residual:    7229.772                F-statistic           :      5.3140
Sigma-square        :      13.903                Prob(F-statistic)     :     0.02155
S.E. of regression  :       3.729                Log likelihood        :   -1426.671
Sigma-square ML     :      13.850                Akaike info criterion :    2857.342
S.E of regression ML:      3.7216                Schwarz criterion     :    2865.857

--------------------------------------------------------------------------

In [89]:
indep=np.asarray(auckReg_df_2006[['Population']])
y=np.asarray(auckReg_df_2006.TotInd_GeogUnits)
y.shape = (len(y), 1)
m2 = ps.model.spreg.OLS(y,indep,name_x=['Population'], name_y='GeogUnits')
print(m2.summary)
        

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :   GeogUnits                Number of Observations:         560
Mean dependent var  :    276.0321                Number of Variables   :           2
S.D. dependent var  :    308.6700                Degrees of Freedom    :         558
R-squared           :      0.0013
Adjusted R-squared  :     -0.0004
Sum squared residual:53188260.573                F-statistic           :      0.7521
Sigma-square        :   95319.463                Prob(F-statistic)     :      0.3862
S.E. of regression  :     308.739                Log likelihood        :   -4003.801
Sigma-square ML     :   94979.037                Akaike info criterion :    8011.602
S.E of regression ML:    308.1867                Schwarz criterion     :    8020.257

-----------------------------------------------------------------------------

In [82]:
y.shape

(560, 1)

In [50]:
type(test)

numpy.ndarray